# Genexpert data exploration and analysis

*Initialize...*

In [1]:
from IPython.display import display, HTML

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
import geopandas as gp

%matplotlib inline
plt.style.use('ggplot')

In [2]:
priority_deptos = [ 5, 12, 10, 9, 11, 18, 19 ]

## Read cohort data

In [3]:
datadf = pd.read_csv(\
"../../../../Outcome Measurement Data/TUBERCULOSIS/GTM - Tx cohort data 2012-2016.csv", index_col = 0)
deptos = gp.read_file("../../../../Covariates and Other Data/GIS/GT-IGN-cartografia_basica-Departamentos.geojson")

In [26]:
temp = datadf[#(datadf.row_name_.isin(["FALLECIDOS"])) & 
              (datadf.col_name.isin(["TOTAL"])) & 
       (datadf.table.isin(["Nuevos Pulmonares BK+", "Nuevos Pulmonares BK-", "Nuevos Pediatricos", 
                          "Nuevos Extrapulmonares", "Nuevos TB/VIH", "Retratamiento"
                          ])) &
       (datadf.deptocode!=0)].\
    groupby(["year", "row_name"]).value.sum().unstack(1)
temp["total"] = temp.sum(axis="columns")
temp

row_name,ABANDONO/PERDIDA_SEGM,CURADOS,FALLECIDOS,FRACASO_TERAPEUTICO,TRANSFERIDOS,TRATAMIENTOS COMPLETOS,total
year,,,,,,,
2012,194.0,1926.0,227.0,49.0,30.0,1221.0,3647.0
2013,252.0,1859.0,244.0,48.0,23.0,1046.0,3472.0
2014,257.0,1829.0,198.0,50.0,3.0,925.0,3262.0
2015,258.0,1911.0,211.0,46.0,8.0,1015.0,3449.0
2016,215.0,1899.0,196.0,42.0,12.0,922.0,3286.0


In [34]:
temp = datadf[(datadf.col_name.isin(["TOTAL"])) & 
       (datadf.table.isin(["Nuevos Pulmonares Genexpert"
                          ])) &
       (datadf.deptocode!=0)].\
    groupby(["year", "row_name"]).value.sum().unstack(1)

In [34]:
temp["total"] = temp.sum(axis="columns")
temp

row_name,ABANDONO/PERDIDA_SEGM,CURADOS,FALLECIDOS,FRACASO_TERAPEUTICO,TRANSFERIDOS,TRATAMIENTOS COMPLETOS,total
year,,,,,,,
2016,15.0,25.0,11.0,1.0,0.0,137.0,189.0


## Read notifications data 

In [44]:
tbnots = pd.read_csv("../../../../Outcome Measurement Data/TUBERCULOSIS/GTM - TB notifications 2012-2017.csv", 
                     encoding="latin-1", index_col = 0)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (4,8,9,10,12,21,22,23,24,25,26,27,28,29,30,31,32,33,35,36,38,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,63,64,65,66,67,69,70,71,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
tbnots.head()

,NOMBRES,DIRECCION,MUNICIPIO,DEPARTAMENTO,SERVICIODESALUD,SEXO,EDAD,RANGOEDAD,PESOLBS,PESOKG,...,CULTIVO_RESULTADO,CULTIVO_FECHA,FECHA,OBSERVACION,RESISTENCIA,FECHARESULTADOPDS,NotificationDate,YearMonth_,DEPTO_CORRECTED,COD_DEPT
1,NaN,NaN,TUCURU,ALTA VERAPAZ,"CS, TUCURU",m,17.0,015 - 019,123,55.90909091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALTA VERAPAZ,16
2,NaN,NaN,TUCURU,ALTA VERAPAZ,"CS, TUCURU",m,49.0,045 - 049,125,56.81818182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALTA VERAPAZ,16
3,NaN,NaN,TUCURU,ALTA VERAPAZ,"CS, TUCURU",m,25.0,025 - 029,97,44.09090909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALTA VERAPAZ,16
4,NaN,NaN,TUCURU,ALTA VERAPAZ,"CS, TUCURU",f,46.0,045 - 049,88,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALTA VERAPAZ,16
5,NaN,NaN,TELEMAN,ALTA VERAPAZ,"CS, TELEMAN",m,47.0,045 - 049,ND,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALTA VERAPAZ,16


In [60]:
#tbnots.METODODX.value_counts(dropna=False)
tbnots["Genexpert"] =  tbnots.METODODX.map(lambda x: "xpert" in str.lower(x) if not pd.isna(x) else False)

In [56]:
tbnots["Year"] = np.floor(tbnots.YearMonth/100)

In [67]:
tbnots[(tbnots.CONDICIONINGRESO == "nuevo") ].groupby(["Year", "Genexpert"]).Genexpert.count().unstack(1).fillna(0)

Genexpert,False,True
Year,,
2012.0,3192.0,0.0
2013.0,3100.0,0.0
2014.0,2964.0,0.0
2015.0,2854.0,361.0
2016.0,2777.0,328.0
2017.0,2872.0,426.0


In [73]:
tbnots[(tbnots.CONDICIONINGRESO == "nuevo") ].groupby(["Year", "Genexpert", "SEXO", "EDAD"])\
    .Genexpert.count().unstack(1).unstack(1).fillna(0).to_excel("temp.xlsx")